In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data

blank = cv2.imread('blank_28x28.png')
b = np.array(blank,dtype=np.float32)
b = np.transpose(b,(2,1,0))
b = torch.from_numpy(b)

num_epochs = 100
learning_rate = 1e-5


from PIL import  Image,ImageDraw
import numpy as np


data1 = []
data2= []

for i in range(20000):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')
    r = np.random.randint(1,101)
    draw = ImageDraw.Draw(im2)
    draw.rectangle([9,12,12,15],fill='red',outline='red')
    data1.append(np.asarray(im2))


for i in range(20000):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')
    r = np.random.randint(1,101)
    draw = ImageDraw.Draw(im2)
    draw.rectangle([21, 12, 24, 15], fill='red', outline='red')
    data2.append(np.asarray(im2))
        
        
y0 = np.array(data1, dtype=np.float32)
y1 = np.array(data2, dtype=np.float32)
y0 = np.transpose(y0, (0, 3, 2, 1))# This to make channels first for CNN
y1 = np.transpose(y1, (0, 3, 2, 1)) 


print(y0.shape)

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)



torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=250,

    shuffle=True,

    num_workers=0,

)

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True), # b, 8, 2, 2
            nn.Conv2d(8,4,3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(4,2,3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)
        )
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1,2)
        x = self.fc(x)
        return x

class auto_decoder(nn.Module):
    def __init__(self):
        super(auto_decoder,self).__init__()
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 3,padding=1)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 3,padding=1)
        
    def forward(self, x):
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
                
        return x
    

model = autoencoder()


def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((fx.t()), fx) / Nsamples

    covg = torch.matmul((gy.t()), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

model_1 = auto_decoder()
optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=learning_rate,
                             weight_decay=1e-5)


for epoch in range(num_epochs):
    for x,y in loader:

        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================
        optimizer.zero_grad()
        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,28,28)))
    print("y:", model(y[0].reshape(1,3,28,28)))

    print("b:", model(b.reshape(1,3,28,28)))



torch.save(model.state_dict(), './conv_autoencoder.pth')

(20000, 3, 28, 28)
epoch [1/100], loss:0.0079
x:  tensor([[-1.7423]], grad_fn=<AddmmBackward>)
y: tensor([[-1.7373]], grad_fn=<AddmmBackward>)
b: tensor([[-1.0791]], grad_fn=<AddmmBackward>)
epoch [2/100], loss:-0.0032
x:  tensor([[-1.6201]], grad_fn=<AddmmBackward>)
y: tensor([[-1.6452]], grad_fn=<AddmmBackward>)
b: tensor([[-0.9950]], grad_fn=<AddmmBackward>)
epoch [3/100], loss:0.0055
x:  tensor([[-1.3238]], grad_fn=<AddmmBackward>)
y: tensor([[-1.1230]], grad_fn=<AddmmBackward>)
b: tensor([[-1.1239]], grad_fn=<AddmmBackward>)
epoch [4/100], loss:-0.0052
x:  tensor([[-1.1748]], grad_fn=<AddmmBackward>)
y: tensor([[-1.2562]], grad_fn=<AddmmBackward>)
b: tensor([[-1.1827]], grad_fn=<AddmmBackward>)
epoch [5/100], loss:-0.0007
x:  tensor([[-1.2076]], grad_fn=<AddmmBackward>)
y: tensor([[-1.3801]], grad_fn=<AddmmBackward>)
b: tensor([[-1.1846]], grad_fn=<AddmmBackward>)
epoch [6/100], loss:-0.0006
x:  tensor([[-1.4735]], grad_fn=<AddmmBackward>)
y: tensor([[-1.3488]], grad_fn=<AddmmBack

epoch [49/100], loss:0.0038
x:  tensor([[-1.1758]], grad_fn=<AddmmBackward>)
y: tensor([[-1.2806]], grad_fn=<AddmmBackward>)
b: tensor([[-0.9734]], grad_fn=<AddmmBackward>)
epoch [50/100], loss:0.0013
x:  tensor([[-0.8723]], grad_fn=<AddmmBackward>)
y: tensor([[-1.8593]], grad_fn=<AddmmBackward>)
b: tensor([[-0.9645]], grad_fn=<AddmmBackward>)
epoch [51/100], loss:0.0006
x:  tensor([[-1.1698]], grad_fn=<AddmmBackward>)
y: tensor([[-1.0857]], grad_fn=<AddmmBackward>)
b: tensor([[-0.9646]], grad_fn=<AddmmBackward>)
epoch [52/100], loss:0.0023
x:  tensor([[-1.3030]], grad_fn=<AddmmBackward>)
y: tensor([[-1.1851]], grad_fn=<AddmmBackward>)
b: tensor([[-0.9446]], grad_fn=<AddmmBackward>)
epoch [53/100], loss:-0.0021
x:  tensor([[-0.9885]], grad_fn=<AddmmBackward>)
y: tensor([[-1.1147]], grad_fn=<AddmmBackward>)
b: tensor([[-0.9674]], grad_fn=<AddmmBackward>)
epoch [54/100], loss:0.0001
x:  tensor([[-1.1928]], grad_fn=<AddmmBackward>)
y: tensor([[-1.0981]], grad_fn=<AddmmBackward>)
b: tensor

epoch [97/100], loss:0.0001
x:  tensor([[-1.4705]], grad_fn=<AddmmBackward>)
y: tensor([[-1.0895]], grad_fn=<AddmmBackward>)
b: tensor([[-1.0032]], grad_fn=<AddmmBackward>)
epoch [98/100], loss:-0.0068
x:  tensor([[-0.9766]], grad_fn=<AddmmBackward>)
y: tensor([[-1.4908]], grad_fn=<AddmmBackward>)
b: tensor([[-1.0225]], grad_fn=<AddmmBackward>)
epoch [99/100], loss:-0.0008
x:  tensor([[-0.9165]], grad_fn=<AddmmBackward>)
y: tensor([[-1.2703]], grad_fn=<AddmmBackward>)
b: tensor([[-1.0184]], grad_fn=<AddmmBackward>)
epoch [100/100], loss:-0.0071
x:  tensor([[-0.8359]], grad_fn=<AddmmBackward>)
y: tensor([[-1.2272]], grad_fn=<AddmmBackward>)
b: tensor([[-1.0155]], grad_fn=<AddmmBackward>)


In [4]:
for i in range(3):
    im2_n = Image.fromarray(noisy,'RGB')
    b = np.asarray(im2_n)
    b = np.array(b, dtype=np.float32)
    b = np.transpose(b,(2,1,0))
    b = torch.from_numpy(b)
    im2_test = Image.fromarray(noisy,'RGB')
    draw1 = ImageDraw.Draw(im2_test)
    draw1.rectangle([5, 9, 8, 12], fill='red', outline='red')
    z = np.asarray(im2_test)
    z = np.array(z, dtype=np.float32)
    z = torch.from_numpy(z)
    print("b:", model(b.reshape(1,3,28,28)))
    print("z: ", model(z.reshape(1,3,28,28)))
    print("x: ", model(x[i].reshape(1,3,28,28)))
    print("y:", model(y[i].reshape(1,3,28,28)))
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("  ")
    p = Image.fromarray(data1[i])
    p.show()
    q = Image.fromarray(data2[i])
    q.show()
    

b: tensor([[-1.2737]], grad_fn=<AddmmBackward>)
z:  tensor([[-1.1946]], grad_fn=<AddmmBackward>)
x:  tensor([[-0.8359]], grad_fn=<AddmmBackward>)
y: tensor([[-1.2272]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[-1.2737]], grad_fn=<AddmmBackward>)
z:  tensor([[-1.1946]], grad_fn=<AddmmBackward>)
x:  tensor([[-0.9861]], grad_fn=<AddmmBackward>)
y: tensor([[-1.1187]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[-1.2737]], grad_fn=<AddmmBackward>)
z:  tensor([[-1.1946]], grad_fn=<AddmmBackward>)
x:  tensor([[-1.0705]], grad_fn=<AddmmBackward>)
y: tensor([[-0.9647]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  


In [5]:
for i in range(20):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')
    b = np.asarray(im2)
    b = np.array(b, dtype=np.float32)
    b = torch.from_numpy(b)
    print("b:", model(b.reshape(1,3,28,28)))
    print("x: ", model(x[i].reshape(1,3,28,28)))
    print("y:", model(y[i].reshape(1,3,28,28)))
    print("*****************************************************")

b: tensor([[-1.2526]], grad_fn=<AddmmBackward>)
x:  tensor([[-0.8359]], grad_fn=<AddmmBackward>)
y: tensor([[-1.2272]], grad_fn=<AddmmBackward>)
*****************************************************
b: tensor([[-1.0042]], grad_fn=<AddmmBackward>)
x:  tensor([[-0.9861]], grad_fn=<AddmmBackward>)
y: tensor([[-1.1187]], grad_fn=<AddmmBackward>)
*****************************************************
b: tensor([[-1.3410]], grad_fn=<AddmmBackward>)
x:  tensor([[-1.0705]], grad_fn=<AddmmBackward>)
y: tensor([[-0.9647]], grad_fn=<AddmmBackward>)
*****************************************************
b: tensor([[-1.0716]], grad_fn=<AddmmBackward>)
x:  tensor([[-0.9789]], grad_fn=<AddmmBackward>)
y: tensor([[-1.1271]], grad_fn=<AddmmBackward>)
*****************************************************
b: tensor([[-2.3029]], grad_fn=<AddmmBackward>)
x:  tensor([[-1.1096]], grad_fn=<AddmmBackward>)
y: tensor([[-1.5563]], grad_fn=<AddmmBackward>)
*****************************************************
b: te